In [ ]:
import math
import cv2 as cv
import numpy as np
import pandas as pd
from glob import glob
from matplotlib import pyplot as plt
from matplotlib.patches import Ellipse

In [ ]:
base_x = 1020
base_y = 160
img    = cv.imread('../dataset/temp_img/000140.png')
RGBimg = cv.cvtColor(img, cv.COLOR_BGR2RGB)
plt.plot([1295-base_x, 1367-base_x], [181-base_y, 361-base_y],color="red")
plt.imshow(RGBimg)
plt.title('dirction')
plt.show()

# Canny 邊緣偵測

In [ ]:
# create a CLAHE object
# def clahe(img):
#     clahe_ = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
#     cl1    = clahe_.apply(img)
#     return cl1


# def equ(img):
#     equ_ = cv.equalizeHist(img)
#     return equ

In [ ]:
def img_precessing(img, medianblur_kernel=21, canny_threshold1=5, canny_threshold2=40):
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)                     # 轉成灰階
    img_blur = cv.medianBlur(img_gray, medianblur_kernel)              # median blur 
    # img_blur = img_gray                                                # no median blur 
    img_edges= cv.Canny(img_blur, canny_threshold1, canny_threshold2)  # canny

    return img_gray, img_blur, img_edges

In [ ]:
def Get_canny_angles(img_paths, medianblur_kernel=21, canny_threshold1=5, canny_threshold2=40):
    canny_pred_angles=[]

    for img_path in img_paths:
        img_name = img_path.split("/")[-1]
        img      = cv.imread(img_path)

        img_gray, blur, edges = img_precessing(img, medianblur_kernel, canny_threshold1, canny_threshold2)
        # print(img_name)
        ###############
        cnt, hierarchy = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        ###### 找全部的輪廓出來畫
        allcnt=[]
        for c in cnt:
            allcnt.extend(c.tolist())
        allcnt = np.array(allcnt)   # 有可能找不到輪廓!!
        if len(allcnt) >= 5:
            
            ####### 找出橢圓
            img_elli = img.copy()
            ellipse = cv.fitEllipseAMS(allcnt)  # fit出橢圓
            # cv.ellipse(img_elli, ellipse, (0,0,255), 3)
            
            ####### 找出角度(11點鐘方向)
            img_11=img.copy()
            (xc, yc), (width, height), angle = ellipse     # (xc,yc):橢圓中心點 (width, height):橢圓寬長
            # print("angle= ", int(angle))
            
            if angle>90:  # 一開始的angle是比較長短邊的角度
                normal_angle = 360-angle-60
                # print('normal angle=', normal_angle)
                
                ######################################## 處理特殊角度(150~180) ###############################################
                if normal_angle > 150 and normal_angle <=180:   
                    normal_angle = normal_angle - 30
                    canny_pred_angles.append((img_name, normal_angle))
                    # rmajor = height / 2
                    # xtop0  = xc + math.cos(math.radians(angle+120)) * rmajor
                    # ytop0  = yc + math.sin(math.radians(angle+120)) * rmajor
                    # rmajor = height / 2
                    # xtop11 = xc + math.cos(math.radians(angle+90)) * rmajor
                    # ytop11 = yc + math.sin(math.radians(angle+90)) * rmajor
                    # rminor = width / 2
                    # xtop = xc + math.cos(math.radians(angle+30)) * rminor
                    # ytop = yc + math.sin(math.radians(angle+30)) * rminor
                    # ellipse_info = Ellipse(xy=(xc, yc), width=width, height=height, angle=angle+30, edgecolor='b', fc='None', lw=2, linestyle='dotted')
                ######################################## 處理特殊角度(180~) ################################################
                elif normal_angle > 180:                         
                    normal_angle = normal_angle - 60
                    canny_pred_angles.append((img_name, normal_angle))
                    # rmajor = height / 2
                    # xtop0  = xc + math.cos(math.radians(angle+150)) * rmajor
                    # ytop0  = yc + math.sin(math.radians(angle+150)) * rmajor
                    # rmajor = height / 2
                    # xtop11 = xc + math.cos(math.radians(angle+120)) * rmajor
                    # ytop11 = yc + math.sin(math.radians(angle+120)) * rmajor
                    # rminor = width / 2
                    # xtop = xc + math.cos(math.radians(angle+60)) * rminor
                    # ytop = yc + math.sin(math.radians(angle+60)) * rminor
                    # ellipse_info = Ellipse(xy=(xc, yc), width=width, height=height, angle=angle+60, edgecolor='b', fc='None', lw=2, linestyle='dotted')
                ######################################## 正常角度(0~150) ################################################
                else:
                    canny_pred_angles.append((img_name, normal_angle))
                    # rmajor = height / 2
                    # xtop0  = xc + math.cos(math.radians(angle+90)) * rmajor
                    # ytop0  = yc + math.sin(math.radians(angle+90)) * rmajor
                    # rmajor = height / 2
                    # xtop11 = xc + math.cos(math.radians(angle+60)) * rmajor
                    # ytop11 = yc + math.sin(math.radians(angle+60)) * rmajor
                    # rminor = width / 2
                    # xtop = xc + math.cos(math.radians(angle)) * rminor
                    # ytop = yc + math.sin(math.radians(angle)) * rminor
                    # ellipse_info = Ellipse(xy=(xc, yc), width=width, height=height, angle=angle, edgecolor='b', fc='None', lw=2, linestyle='dotted')

                # print('new normal angle=', normal_angle)
                # cv.line(img_11, (int(xtop0), int(ytop0)), (int(xc), int(yc)), (0, 0, 255), 1)
                # cv.line(img_11, (int(xtop11), int(ytop11)), (int(xc), int(yc)), (128, 0, 0), 3)
                # cv.line(img_11, (int(xtop), int(ytop)), (int(xc), int(yc)), (0, 0, 255), 1)
                


            if angle<90:
                normal_angle = 180-angle-60
                # print('normal angle=', normal_angle)
                

                ######################################## 處理特殊角度(0~30) ###############################################  
                if normal_angle > 0 and normal_angle <= 30:   
                    normal_angle = normal_angle + 60                            ## !!!!!!!!!!!!!
                    canny_pred_angles.append((img_name, normal_angle))
                    # rmajor = height / 2
                    # xtop0  = xc + math.cos(math.radians(angle+210)) * rmajor
                    # ytop0  = yc + math.sin(math.radians(angle+210)) * rmajor
                    # rmajor = height / 2
                    # xtop11 = xc + math.cos(math.radians(angle+180)) * rmajor
                    # ytop11 = yc + math.sin(math.radians(angle+180)) * rmajor
                    # rminor = width / 2
                    # xtop = xc + math.cos(math.radians(angle+120)) * rminor
                    # ytop = yc + math.sin(math.radians(angle+120)) * rminor
                    # ellipse_info = Ellipse(xy=(xc, yc), width=width, height=height, angle=angle+120, edgecolor='b', fc='None', lw=2, linestyle='dotted')
                ######################################## 處理特殊角度(30~60) ###############################################
                elif normal_angle > 30 and normal_angle <= 60:   
                    normal_angle = normal_angle + 30                            ## !!!!!!!!!!!!!!
                    canny_pred_angles.append((img_name, normal_angle))
                    # rmajor = height / 2
                    # xtop0  = xc + math.cos(math.radians(angle+240)) * rmajor
                    # ytop0  = yc + math.sin(math.radians(angle+240)) * rmajor
                    # rmajor = height / 2
                    # xtop11 = xc + math.cos(math.radians(angle+210)) * rmajor
                    # ytop11 = yc + math.sin(math.radians(angle+210)) * rmajor
                    # rminor = width / 2
                    # xtop = xc + math.cos(math.radians(angle+150)) * rminor
                    # ytop = yc + math.sin(math.radians(angle+150)) * rminor
                    # ellipse_info = Ellipse(xy=(xc, yc), width=width, height=height, angle=angle+150, edgecolor='b', fc='None', lw=2, linestyle='dotted')
                ######################################## 正常角度(0~150) ################################################
                else:
                    canny_pred_angles.append((img_name, normal_angle))
                    # rmajor = height / 2
                    # xtop0  = xc + math.cos(math.radians(angle+270)) * rmajor
                    # ytop0  = yc + math.sin(math.radians(angle+270)) * rmajor
                    # rmajor = height / 2
                    # xtop11 = xc + math.cos(math.radians(angle+240)) * rmajor
                    # ytop11 = yc + math.sin(math.radians(angle+240)) * rmajor
                    # rminor = width / 2
                    # xtop = xc + math.cos(math.radians(angle+180)) * rminor
                    # ytop = yc + math.sin(math.radians(angle+180)) * rminor
                    # ellipse_info = Ellipse(xy=(xc, yc), width=width, height=height, angle=angle+180, edgecolor='b', fc='None', lw=2, linestyle='dotted')
                
                # print('new normal angle=', normal_angle)
                # cv.line(img_11, (int(xtop0), int(ytop0)), (int(xc), int(yc)), (0, 0, 255), 1)
                # cv.line(img_11, (int(xtop11), int(ytop11)), (int(xc), int(yc)), (128, 0, 0), 3)
                # cv.line(img_11, (int(xtop), int(ytop)), (int(xc), int(yc)), (0, 0, 255), 1)

            
            ################# plot img
            # plt.figure()
            # f, axarr = plt.subplots(1, 5,figsize=(20,8)) 
            # axarr[0].imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
            # axarr[1].imshow(cv.cvtColor(img_gray, cv.COLOR_BGR2RGB))
            # axarr[2].imshow(cv.cvtColor(blur, cv.COLOR_BGR2RGB))
            # axarr[3].imshow(cv.cvtColor(edges, cv.COLOR_BGR2RGB))
            # axarr[4].imshow(cv.cvtColor(img_11, cv.COLOR_BGR2RGB))
            # plt.show()
            ##################### save img
            # plt.imsave("../dataset/training_dataset/img/train/Crop_Gray/{}".format(img_name), cv.cvtColor(img_gray, cv.COLOR_BGR2RGB))       # !!!!!!!!!!!!
            # plt.imsave("../dataset/training_dataset/img/train/Crop_MedianBlur/{}".format(img_name), cv.cvtColor(blur, cv.COLOR_BGR2RGB))     # !!!!!!!!!!!!
            # plt.imsave("../dataset/training_dataset/img/train/Crop_CannyEdge/{}".format(img_name), cv.cvtColor(edges, cv.COLOR_BGR2RGB))     # !!!!!!!!!!!!
            # plt.imsave("../dataset/result_img/canny_traindata_direction_result/canny_{}".format(img_name), cv.cvtColor(img_11, cv.COLOR_BGR2RGB))  # !!!!!!!!!!!!

            # plt.figure()
            # f, ax = plt.subplots(figsize=(3,4)) 
            # ax.add_artist(ellipse_info)  # 橢圓
            # ax.set_axis_off()
            # plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
            # plt.savefig("../dataset/result_img/canny_traindata_ellipse_result/ellipse_{}".format(img_name),bbox_inches='tight', pad_inches=0)    # !!!!!!!!!!!!
            # plt.close(f)
        


    return canny_pred_angles

In [ ]:
def Get_MAE(canny_pred_angles, true_df):
    pred_df = pd.DataFrame(canny_pred_angles,columns=['imgname','pred_angle'])

    ''' true data left join '''
    true_df_all = pd.merge(true_df, pred_df,  how='left', left_on=['imgname'], right_on = ['imgname'])

    true_df_all['angle_error'] = abs(true_df_all['true_angle']-true_df_all['pred_angle'])
    
    mae = round(true_df_all['angle_error'].mean(),3)
    print("mean angle error= ", mae)
    
    pmae = mae/180
    print("percentage mean angle error= ", pmae)
    return true_df_all, mae, pmae

In [ ]:
training_img_paths = sorted(glob('../dataset_20230323/training_dataset/cropped_img/*.png'))                  # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! 換資料集

'''use true_angle'''
training_trueangle_df = pd.read_csv('../dataset_20230323/training_dataset/direc/true_angle_20230323.csv') 


# 找到最佳參數組合

In [ ]:
train_mae_list=[]
for medianblur_kernel in range(9,34,4):        
    for canny_threshold1 in range(5,26,5):       
        for canny_threshold2 in range(35,56,5): 
            print('========================',medianblur_kernel, canny_threshold1, canny_threshold2,'===========================')
            canny_pred_angles      = Get_canny_angles(training_img_paths, medianblur_kernel, canny_threshold1, canny_threshold2)
            true_df_all, mae, pmae = Get_MAE(canny_pred_angles, training_trueangle_df)
            train_mae_list.append((medianblur_kernel, canny_threshold1, canny_threshold2, mae, pmae))

In [ ]:
# 結果

min_mae = 100
min_result = []
for mae in train_mae_list:
    if mae[3]<min_mae:
        min_mae = mae[3]
        min_result = mae

print("最佳參數組合 = ", min_result)

In [ ]:
# import pickle
# with open("../dataset/result_mae/train/train_mae_result_20221222.pk", 'wb') as f:
#     pickle.dump(train_mae_list, f)

In [ ]:
# min_i = 0
# min_mae = train_mae_list[0][3]

# for i in range(1, len(train_mae_list)):
#     if train_mae_list[i][3] < min_mae:
#         min_mae = train_mae_list[i][3]
#         min_i = i

# print(train_mae_list[min_i])

In [ ]:
# 測試集結果
# test_img_paths = sorted(glob('../dataset/test_dataset/img/Crop/*.png'))
# tes_df         = pd.read_csv('../dataset/test_dataset/direc/old_angle_20221222.csv') 

# canny_pred_angles = Get_canny_angles(test_img_paths, medianblur_kernel=9, canny_threshold1=5, canny_threshold2=40)
# true_df_all, mae, mpae = Get_MAE(canny_pred_angles, tes_df)

# 在圖畫方向

In [ ]:
# img_names = cannytes_df['img_name'].values
# img_names[:5]

In [ ]:
# img_name = 'l_000058.png'
# img_name = 'h_000081.png'

# for img_name in img_names:

#     canny_angle = cannytes_df[cannytes_df['img_name']==img_name]['canny_angle'].values[0]
#     print('canny_angle= ',canny_angle)

#     # 把方向畫在左上角落
#     bbox_xc = (100+100)/2  
#     bbox_yc = (100+100)/2

#     xtop11 = bbox_xc + math.cos(math.radians(canny_angle)) * 100
#     ytop11 = bbox_yc - math.sin(math.radians(canny_angle)) * 100

#     uncrop_img = cv.imread(uncrop_img_path + img_name)
#     img_rgb    = cv.cvtColor(uncrop_img, cv.COLOR_BGR2RGB)  # 轉成RGB


#     plt.figure()
#     f, ax = plt.subplots() 
#     plt.plot([0, 200], [100, 100], color="gray", linestyle="--")   # 水平線
#     plt.plot([100, 100], [0, 100], color="gray", linestyle="--")   # 重直線
#     plt.plot([xtop11, bbox_xc], [ytop11, bbox_yc], color="r") # 11點方向線
#     plt.text(100, 100, int(canny_angle), ha='center', va='top', fontsize=10, color='white')
    
#     ax.set_axis_off()
#     plt.imshow(img_rgb)       # [x1:x2 ,y1:y2]
    
#     plt.savefig("../dataset/result_img/canny_testdata_upCrop_direction_result/direction_{}".format(img_name),bbox_inches='tight', pad_inches=0)
#     plt.close(f)



In [ ]:
# img_paths = sorted(glob('../dataset/training_dataset/img/train/Crop/*.png'))
# img_paths = sorted(glob('../dataset/training_dataset/img/validation/Crop/*.png'))
img_paths = sorted(glob('../dataset/test_dataset/img/Crop/*.png'))


for img_path in img_paths[2:4]:
    img_name = img_path.split("/")[-1].split(".")[-2]
    img      = cv.imread(img_path)
    img_rgb  = cv.cvtColor(img, cv.COLOR_BGR2RGB)  # 轉成灰階
    # img_equ  = equ(img)
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)  # 轉成灰階
    # clahe = clahe(img)
    blur_G  = cv.GaussianBlur(img_gray, (21, 21), 0)
    blur_M  = cv.medianBlur(img_gray, 21)
    # img   = cv.bilateralFilter(img, 15, 50,  50)
    blur_B  = cv.bilateralFilter(img_gray, 21, 50,  50)
    
    
    # canny
    # edges = cv.Canny(blur, 5, 30)

    # sobel
    x    = cv.Sobel(blur_M,cv.CV_16S,1,0)
    y    = cv.Sobel(blur_M,cv.CV_16S,0,1)
    absX = cv.convertScaleAbs(x)# 轉回uint8
    absY = cv.convertScaleAbs(y)
    dst  = cv.addWeighted(absX, 0.5, absY, 0.5, 0)
    # threshold
    # edges = scale(y, 20)
    # ret, th1 = cv.threshold(dst, 50, 255, cv.THRESH_BINARY)
    # th2 = cv.adaptiveThreshold(dst, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 11, 2)
    # th3 = cv.adaptiveThreshold(dst, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11, 2)
    # edges = cv.addWeighted(absX,0.5,absY,0.5,0)

    # edges  = cv.medianBlur(edges, 3)
    edges_M = cv.Canny(blur_M, 5, 40)
    edges_G = cv.Canny(blur_G, 5, 40)
    edges_B = cv.Canny(blur_B, 5, 40)
    print(img_name)

    #### Save npz
    # np.save('../dataset/test_dataset/img/Crop_Edge/{}'.format(img_name), edges)
    # np.save('../dataset/training_dataset/img/validation/Crop_Edge/{}'.format(img_name), edges)
    # np.save('../dataset/training_dataset/img/train/Crop_Edge/{}'.format(img_name), edges)
    
    # plt.imsave('../dataset/test_dataset/img/Crop_Edge/{}'.format(img_name),edges)
    # plt.imshow(img,cmap='gray')
    # plt.title('Original Image')
    # plt.show()
    # plt.imshow(equ,cmap='gray')
    # plt.title('equalizeHist Image')
    # plt.show()

    # plt.imshow(blur,cmap='gray')
    # plt.title('blur Image')
    # plt.show()
    

    #subplot(r,c) provide the no. of rows and columns
    
    ###############
    # cnt, hierarchy = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    # # 找最大的輪廓出來畫
    # max_i = 0
    # max_cnt = len(cnt[0])
    # for i in range(1, len(cnt)):
    #     if len(cnt[i])>max_cnt:
    #         max_cnt=len(cnt[i])
    #         max_i=i


    # 畫線
    # rows,cols = img.shape[:2]
    # [vx,vy,x,y] = cv.fitLine(cnt, cv.DIST_L2,0,0.01,0.01)
    # lefty = int((-x*vy/vx) + y)
    # righty = int(((cols-x)*vy/vx)+y)
    # cv.line(img,(cols-1,righty),(0,lefty),(0,255,0),2)

    # cv.drawContours(img, cnt, -1, 255, 3)
    # find the biggest countour (c) by the area
    # c = max(cnt, key = cv.contourArea)
    # x, y, w, h = cv.boundingRect(c)

    # draw the biggest contour (c) in green
    # cv.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    # hull = cv.convexHull(cnt[max_i])
    # length = len(hull)
    # for i in range(length):
    #     cv.line(img, tuple(hull[i][0]), tuple(hull[(i+1)%length][0]), (0,0,255), 2)
    # 绘制椭圆
    # ellipse = cv.fitEllipse(cnt[max_i])
    # cv.ellipse(img,ellipse,(0,0,255),10)
    ###############

    ###############
    plt.figure()
    f, axarr = plt.subplots(1, 5,figsize=(15,3)) 
    axarr[0].imshow(img_rgb)
    axarr[1].imshow(cv.cvtColor(dst, cv.COLOR_BGR2RGB))
    axarr[2].imshow(cv.cvtColor(edges_G, cv.COLOR_BGR2RGB))
    axarr[3].imshow(cv.cvtColor(edges_M, cv.COLOR_BGR2RGB))
    axarr[4].imshow(cv.cvtColor(edges_B, cv.COLOR_BGR2RGB))
    plt.show()
    #####################